In [ ]:
import pandas as pd
import re

In [ ]:
dataset = "./ABBYY_clear.txt"

In [ ]:
with open(dataset, "r") as f: # reading a txt file
    data = f.readlines()

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df[['word', 'description']] = df[0].str.extract(r'^([^(=\[\-,]+?)(?:\s*[\(=\[\-,]\s*(.*))?$', expand=True)
df = df.drop(0, axis=1)

In [ ]:
step_0 = df
step_0.info()

In [ ]:
# Cleaning
#########################
df.drop_duplicates(inplace=True)
df = df[~df["word"].str.contains(r'-\d+-', regex=True, na=False)] # -number- deletion
df['description'] = df['description'].fillna('N/A')
#########################

In [ ]:
step_1 = df
step_1.info()

In [ ]:
df = df.dropna(subset=['word']) # NaN for word section

In [ ]:
df[df['description'] == 'N/A'].shape[0] # 2044 left, probably to drop, most of them are just page initials
df.drop(df[df['description'] == 'N/A'].index, inplace=True)

In [ ]:
df.info()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("ZiartisNikolas/NMT-cypriot-dialect-to-greek")
model = AutoModelForSeq2SeqLM.from_pretrained("ZiartisNikolas/NMT-cypriot-dialect-to-greek")

In [ ]:
# !pip install sacremoses # recommended for model

In [ ]:
def translate_to_greek(text):
    global translation_counter, df

    if pd.isna(text) or text == '':
        return ''

    if translation_counter < 10:
        print(f"Input: {text[:100]}...")

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=200)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, num_beams=4, early_stopping=True)

    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)


    return translation

In [ ]:
tqdm.pandas(desc="Translating")

df['greek_word'] = df['word'][15000:].progress_apply(translate_to_greek)
# df['greek_description'] = df['description'].progress_apply(translate_to_greek)

# print(df[['word', 'description', 'greek_translation']].head())